In [1]:
import pandas as pd
import openpyxl
# packages

# Load and Calculate Inflation Data

In [10]:
# Specify the dynamic year you want to use as the baseline
dynamic_year = 1983  # Change this to the desired year min 1983 max 2022

In [13]:
import pandas as pd

# Replace 'your_file.xlsx' with the path to your Excel file
file_path = 'data/su-e-05.02.67.xlsx'

# Replace 'INDEX_y' with the name of the sheet you want to read
sheet_name = 'VAR_y-1'

# Read the Excel file with the specified header row
data = pd.read_excel(file_path, sheet_name=sheet_name, header=3)

# Last valid entry in excel
last_valid_entry = 415  # You can adjust this value based on your data

# Set the dynamic year values to 100 and remove columns before that year
df = data.iloc[:last_valid_entry + 1]  # This will keep rows up to the last_valid_entry

In [14]:
df

,Code,PosNo,PosType,Level,COICOP,Position_D,PosTxt_D,Position_F,PosTxt_F,Posizione_I,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,100_100,100.0,1.0,1.0,NaN,Total,Total,Total,Total,Totale,...,-0.2,0,-1.1,-0.4,0.5,0.9,0.4,-0.7,0.6,2.8
1,100_1,1.0,2.0,2.0,'01,Nahrungsmittel und alkoholfreie Getränke,Nahrungsmittel und alkoholfreie Getränke,Alimentation et boissons non alcoolisées,Alimentation et boissons non alcoolisées,Prodotti alimentari e bevande analcoliche,...,1.2,0.9,-0.8,0.4,0.4,1.3,0.1,0.1,-1.6,1.7
2,100_1001,1001.0,3.0,3.0,'01.1,Nahrungsmittel,Nahrungsmittel,Alimentation,Alimentation,Prodotti alimentari,...,1.4,1.1,-0.8,0.5,0.5,1.3,0.2,0.1,-1.6,1.6
3,100_1002,1002.0,3.0,4.0,'01.1.1,"Brot, Mehl und Getreideprodukte","Brot, Mehl und Getreideprodukte","Pain, farine et produits céréaliers","Pain, farine et produits céréaliers","Pane, farina e cereali",...,-0.6,0.1,0.4,-0.1,-0.2,0.8,0,-0.8,-0.9,2.7
4,100_1003,1003.0,4.0,5.0,'01.1.1.1,Reis,Reis,Riz,Riz,Riso,...,2.5,13.8,1,1.1,-1.2,-0.6,2.6,-0.6,-2.2,1.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
411,100_12536,12536.0,4.0,4.0,NaN,Gebühren für Wertschriftendepots,Gebühren für Wertschriftendepots,Frais pour les dépôts de titres,Frais pour les dépôts de titres,Spese deposito titoli,...,...,...,...,...,3.4,0,1.6,2.4,1.5,2.4
412,100_12541,12541.0,3.0,3.0,'12.7,Sonstige Dienstleistungen,Sonstige Dienstleistungen,Autres services,Autres services,Altri servizi,...,1.5,0.9,0.2,0.1,1.4,4.2,12.2,3.5,-1.2,1.7
413,100_12543,12543.0,4.0,4.0,'12.7.0.1,Amtsdokumente,Amtsdokumente,Documents officiels,Documents officiels,Documenti ufficiali,...,...,...,...,...,...,...,...,...,...,0.0
414,100_12547,12547.0,4.0,4.0,'12.7.0.2,Dienste von Rechtsanwälten,Dienste von Rechtsanwälten,Services d'avocats,Services d'avocats,Servizi di avvocati,...,...,...,...,...,...,...,...,...,...,0.2


In [11]:

import pandas as pd

# Replace 'your_file.xlsx' with the path to your Excel file
file_path = 'data/su-e-05.02.67.xlsx'

# Replace 'INDEX_y' with the name of the sheet you want to read
sheet_name = 'VAR_y-1'

# Read the Excel file with the specified header row
data = pd.read_excel(file_path, sheet_name=sheet_name, header=3)

# Last valid entry in excel
last_valid_entry = 415  # You can adjust this value based on your data

# Set the dynamic year values to 100 and remove columns before that year
df = data.iloc[:last_valid_entry + 1]  # This will keep rows up to the last_valid_entry

df_2 = df[df["Level"] == 2]

# Set the dynamic year values to 100
df_2.loc[:, dynamic_year] = 100

# Loop through years and calculate real values and subtract 100, starting from the dynamic year
for year in range(dynamic_year + 1, 2023):
    df_2[year] = df_2[year - 1] * (1 + df_2[year] / 100)

for year in range(dynamic_year, 2023):
    df_2[year] = df_2[year] - 100

subset_columns = ['PosTxt_E', 'PosNo', dynamic_year] + list(range(dynamic_year + 1, 2023))
df_inflation = df_2[subset_columns]
df_inflation = df_inflation.rename(columns={'PosNo': 'ID'})


/var/folders/0q/zp8d2_c16z32wsh2hdjb9fch0000gn/T/ipykernel_27610/637106177.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2[year] = df_2[year - 1] * (1 + df_2[year] / 100)
/var/folders/0q/zp8d2_c16z32wsh2hdjb9fch0000gn/T/ipykernel_27610/637106177.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2[year] = df_2[year] - 100


In [12]:
csv_file_path = "data/df_inflation.csv"  # Replace "your_file_path_here.csv" with the desired file path
df_inflation.to_csv(csv_file_path, index=False)  # Setting index=False to exclude the index column from the CSV

# Load inflation basket composition

In [7]:
valid_gp_values = [1,2,3,4,5,6,7,8,9,10,11,12]

def process_excel_data(file_path,
                       sheet_name,
                       header_row,
                       last_valid_entry,
                       valid_column,
                       year_columns,
                       id_column,
                       id_column_name,
                       additional_columns=None):
    
    data = pd.read_excel(file_path, sheet_name=sheet_name, header=header_row)
    data = data.iloc[:last_valid_entry + 1]

    filtered_data = data[data[valid_column].isin(valid_gp_values)]
    selected_columns = additional_columns + [valid_column] + year_columns
    filtered_data = filtered_data[selected_columns]
    filtered_data = filtered_data.rename(columns={valid_column: id_column_name})

    return filtered_data

file_path = 'data/su-d-05.02.90.xlsx'

# Process 2020 data
df_b_l2_20 = process_excel_data(file_path, 'LIK2020', 3, 415, 'PosNo', [2021, '2022', '2023'], 'PosNo', 'ID', additional_columns=['PosTxt_E'])

# Process 2015 data
valid_PosNo_15 = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
df_b_l2_15 = process_excel_data(file_path, 'LIK2015', 3, 396, 'PosNo', [2016, 2017, '2018', '2019', '2020'], 'PosNo', 'ID', additional_columns=['PosTxt_E'])

# Process 2010 data
valid_gp_values_10 = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
df_b_l2_10 = process_excel_data(file_path, 'LIK2010', 3, 313, 'GP Nr.', [2011, 2012, 2013, 2014, 2015], 'GP Nr.', 'ID', additional_columns=['deutsch'])

# Process 2005 data
valid_gp_values_05 = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
df_b_l2_05 = process_excel_data(file_path, 'LIK2005', 3, 313, 'GP Nr.', [2006, 2007, 2008, 2009, 2010], 'GP Nr.', 'ID', additional_columns=['deutsch'])

# Process 2000 data
valid_gp_values_00 = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
df_b_l2_00 = process_excel_data(file_path, 'LIK2000', 3, 313, 'Nr. ', ['2000/01', 2002, 2003, 2004, 2005], 'Nr. ', 'ID', additional_columns=['Position'])


In [8]:
# Start with df_b_l2_00 and perform left joins in the reverse order
left_merged_df = pd.merge(df_b_l2_00, df_b_l2_05, on='ID', how='left')
left_merged_df = pd.merge(left_merged_df, df_b_l2_10, on='ID', how='left')
left_merged_df = pd.merge(left_merged_df, df_b_l2_15, on='ID', how='left')
left_merged_df = pd.merge(left_merged_df, df_b_l2_20, on='ID', how='left')
left_merged_df.columns = left_merged_df.columns.astype(str)
df_weights = left_merged_df[['Position','ID','2000/01','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015','2016','2017','2018','2019','2020','2021','2022','2023']]
df_weights = df_weights.rename(columns={'2000/01': '2001'})
columns_to_convert = df_weights .columns[2:]  # Assuming the percentage columns start from the 3rd column
df_weights[columns_to_convert] = df_weights [columns_to_convert] / 100 # convert to decimal

csv_file_path = "data/df_weights.csv"  # Replace "your_file_path_here.csv" with the desired file path
df_weights.to_csv(csv_file_path, index=False)  # Setting index=False to exclude the index column from the CSV